In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

model_path = '/content/drive/MyDrive/best_segmentation0.1v.pt'

if os.path.exists(model_path):
    print("✅ Model file found!")
else:
    raise FileNotFoundError(f"❌ Model file NOT found at: {model_path}")


Mounted at /content/drive
✅ Model file found!


In [ ]:
!pip install ultralytics opencv-python numpy --quiet


In [ ]:
from ultralytics import YOLO
import cv2

model = YOLO(model_path)

video_path = '/content/drive/MyDrive/test_video.mp4'

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise FileNotFoundError(f"❌ Cannot open video: {video_path}")

fps = cap.get(cv2.CAP_PROP_FPS) or 25
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

model_path = '/content/drive/MyDrive/best_segmentation0.1v.pt'
video_path = '/content/drive/MyDrive/test_video.mp4'
output_path = '/content/drive/MyDrive/bubble_segmentation_output_2.mp4'

#Confirm files exist
for p in [model_path, video_path]:
    if not os.path.exists(p):
        raise FileNotFoundError(f"❌ File not found: {p}")
print("✅ Model & input video found!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model & input video found!


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load YOLO model
model = YOLO(model_path)

# Open video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise FileNotFoundError(f"❌ Cannot open video: {video_path}")

fps = cap.get(cv2.CAP_PROP_FPS) or 25
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # mp4v is safest for Colab + Drive
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))


In [ ]:
total_bubbles = 0
frame_number = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_number += 1

    results = model.predict(source=[frame], conf=0.5, iou=0.4, verbose=False)
    frame_bubble_count = 0

    if results[0].masks is not None:
        masks = results[0].masks.xy
        frame_bubble_count = len(masks)
        total_bubbles += frame_bubble_count

        for mask_points in masks:
            mask_points = np.array(mask_points, dtype=np.int32)
            area = cv2.contourArea(mask_points)
            radius_px = np.sqrt(area / np.pi) if area > 0 else 0
            cv2.polylines(frame, [mask_points], True, (0, 255, 0), 2)
            x, y = mask_points[0]
            cv2.putText(frame, f"R:{radius_px:.1f}px", (int(x), int(y)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    cv2.putText(frame, f"Frame: {frame_number}", (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, f"Frame Bubbles: {frame_bubble_count}", (20, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"Total Bubbles: {total_bubbles}", (20, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
print(f"✅ Video saved to: {output_path}")


✅ Video saved to: /content/drive/MyDrive/bubble_segmentation_output_2.mp4


In [ ]:
import time

time.sleep(2)  # wait for Drive sync
if os.path.exists(output_path):
    print("✅ Output video found:", output_path)
else:
    raise FileNotFoundError("❌ Output video was not saved!")


✅ Output video found: /content/drive/MyDrive/bubble_segmentation_output_2.mp4


In [ ]:
from IPython.display import HTML
from base64 import b64encode

with open(output_path, 'rb') as f:
    mp4 = f.read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=700 controls><source src="{data_url}" type="video/mp4"></video>')


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import time

time.sleep(2)  # wait for Drive sync
if os.path.exists(output_path):
    print("✅ Output video found:", output_path)
else:
    raise FileNotFoundError("❌ Output video was not saved!")


In [ ]:
import numpy as np

output_path = '/content/drive/MyDrive/bubble_segmentation_output_2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'avc1')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

total_bubbles = 0
frame_number = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_number += 1

    # Predict
    results = model.predict(source=[frame], conf=0.5, iou=0.4, verbose=False)
    frame_bubble_count = 0

    if results[0].masks is not None:
        masks = results[0].masks.xy
        frame_bubble_count = len(masks)
        total_bubbles += frame_bubble_count

        for mask_points in masks:
            mask_points = np.array(mask_points, dtype=np.int32)
            area = cv2.contourArea(mask_points)
            radius_px = np.sqrt(area / np.pi) if area > 0 else 0
            cv2.polylines(frame, [mask_points], True, (0, 255, 0), 2)
            x, y = mask_points[0]
            cv2.putText(frame, f"R:{radius_px:.1f}px", (int(x), int(y)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    # Overlay text
    cv2.putText(frame, f"Frame: {frame_number}", (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, f"Frame Bubbles: {frame_bubble_count}", (20, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"Total Bubbles: {total_bubbles}", (20, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
print(f"✅ Processing complete! Video saved at: {output_path}")


✅ Processing complete! Video saved at: /content/drive/MyDrive/bubble_segmentation_output_2.mp4


In [ ]:
for mask_points in masks:
    mask_points = np.array(mask_points, dtype=np.int32)

    # Calculate area and radius
    area = cv2.contourArea(mask_points)
    radius_px = np.sqrt(area / np.pi) if area > 0 else 0

    # Get center of mask (better than first point)
    M = cv2.moments(mask_points)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
    else:
        cx, cy = int(mask_points[:, 0, 0].mean()), int(mask_points[:, 0, 1].mean())

    # Draw green contour
    cv2.polylines(frame, [mask_points], True, (0, 255, 0), 2)

    # Draw radius label near center
    cv2.putText(frame, f"R: {radius_px:.1f}px",
                (cx, cy - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

In [ ]:
if results[0].boxes.is_track:
    track_ids = results[0].boxes.id.int().cpu().tolist()
    unique_bubbles_in_frame = len(set(track_ids))
    total_unique_bubbles = max(track_ids) if track_ids else total_unique_bubbles